<a href="https://colab.research.google.com/github/HrithikVaya007/Churn-Predictor-App/blob/main/churn_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
code = """
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# Set page layout and title
st.set_page_config(layout="wide")
st.title("!!!Universal Customer Churn Predictor Just Upload any Churn File in .csv format!!!")

# Allow user to upload a CSV file
uploaded_file = st.file_uploader(" Upload your dataset (CSV)", type=["csv"])

if uploaded_file:
    # Load the uploaded CSV file into a DataFrame
    df = pd.read_csv(uploaded_file)

    # Try to clean common numeric columns that may be wrongly typed as strings
    for col in ['TotalCharges', 'MonthlyCharges']:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # Show the first few rows of the dataset
    st.write(" Dataset Preview:", df.head())

    # Try to automatically detect a churn/target column
    binary_candidates = [col for col in df.columns if df[col].nunique() == 2]
    churn_name_matches = [col for col in df.columns if any(key in col.lower() for key in ['churn', 'exited', 'target'])]
    default_churn = churn_name_matches[0] if churn_name_matches else (binary_candidates[0] if binary_candidates else None)

    if not default_churn:
        st.error(" Could not detect a binary churn column. Please ensure your dataset has a 0/1 or Yes/No target column.")
        st.stop()

    # Let user confirm or choose the churn column
    churn_column = st.selectbox(" Select churn column (binary)", options=binary_candidates, index=binary_candidates.index(default_churn))

    # Convert 'Yes'/'No' or '1'/'0' text values to binary 1/0
    y_raw = df[churn_column]
    if y_raw.dtype == 'O':
        y = y_raw.astype(str).str.strip().str.lower().map({'yes': 1, 'no': 0, '1': 1, '0': 0})
    else:
        y = y_raw.copy()

    # If everything became NaN after mapping, show an error
    if y.isna().all():
        st.error(" Churn column could not be interpreted as binary (Yes/No or 0/1). Please check your data.")
        st.stop()

    # Get numeric feature columns, excluding obvious ID fields and the churn column
    numeric_columns = df.select_dtypes(include=[np.number]).columns
    feature_candidates = [col for col in numeric_columns if col.lower() not in ['seniorcitizen', 'customerid'] and col != churn_column]

    if not feature_candidates:
        st.error(" No valid numeric feature columns found for prediction.")
        st.stop()

    # Let user select a numeric column to use as input feature
    default_feature = 'MonthlyCharges' if 'MonthlyCharges' in feature_candidates else feature_candidates[0]
    feature_column = st.selectbox(" Select numeric feature for prediction", options=feature_candidates, index=feature_candidates.index(default_feature))

    # Let user select the degree of the polynomial regression
    degree = st.slider(" Select polynomial degree", 1, 5, 2)

    # Drop rows where either feature or churn is missing
    mask = (~y.isna()) & (~df[feature_column].isna())
    if mask.sum() == 0:
        st.error("⚠ No valid rows found after cleaning. Please check your selected columns.")
        st.stop()

    # Prepare training data
    X = df.loc[mask, [feature_column]]
    y_clean = y[mask]

    # Create polynomial features from input
    poly = PolynomialFeatures(degree=degree)
    X_poly = poly.fit_transform(X)

    # Fit a linear regression model on polynomial features
    model = LinearRegression()
    model.fit(X_poly, y_clean)
    y_pred = model.predict(X_poly)

    # Evaluate and display model performance
    mse = mean_squared_error(y_clean, y_pred)
    st.success(f" Model trained! MSE: {mse:.4f}")

    # Show prediction results
    df_result = df.loc[mask].copy()
    df_result['Predicted Churn Probability'] = y_pred
    st.dataframe(df_result[[feature_column, churn_column, 'Predicted Churn Probability']])

    # Plot actual vs predicted values
    plt.figure(figsize=(8, 5))
    plt.scatter(X, y_clean, color='blue', label='Actual')
    plt.scatter(X, y_pred, color='red', label='Predicted')
    plt.xlabel(feature_column)
    plt.ylabel('Churn Probability')
    plt.legend()
    st.pyplot(plt)
"""



In [48]:

with open("app.py", "w") as f:
    f.write(code)

!pkill -f streamlit
!pkill -f ngrok
PORT = 7860
!streamlit run app.py --server.port {PORT} &>/dev/null &
from pyngrok import ngrok
public_url = ngrok.connect(PORT)
print(f" Your app is live at: {public_url}")

 Your app is live at: NgrokTunnel: "https://9eb2cf244629.ngrok-free.app" -> "http://localhost:7860"


In [49]:
!pip install streamlit pyngrok scikit-learn matplotlib pandas -q